# NBA simulation central

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

nba = pd.read_csv('2017-2018.csv', sep=';')
teams_abb = pd.read_csv('data/Team Abbrev.csv')
teams_abb = teams_abb[teams_abb['season'] == 2018]
teams_abb = teams_abb[['team', 'abbreviation']]

## Creating team coefficients

In [ ]:
# Create an empty dictionary to store team statistics
teams_dict = {
    'Team': [],
    'Wins': [],
    'Losses': [],
    'OFFH': [],
    'DEFH': [],
    'OFFA': [],
    'DEFA': [],
    'OFFHsd': [],
    'DEFHsd': [],
    'OFFAsd': [],
    'DEFAsd': []
}

# Calculate statistics for each team
for team in nba['HomeTeam'].unique():
    team_home = nba[nba['HomeTeam'] == team]
    team_away = nba[nba['AwayTeam'] == team]
    
    # Calculate Home and Away statistics
    home_offense_mean = team_home['HomePTS'].mean()
    home_defense_mean = team_home['AwayPTS'].mean()
    away_offense_mean = team_away['AwayPTS'].mean()
    away_defense_mean = team_away['HomePTS'].mean()
    
    home_offense_std = team_home['HomePTS'].std()
    home_defense_std = team_home['AwayPTS'].std()
    away_offense_std = team_away['AwayPTS'].std()
    away_defense_std = team_away['HomePTS'].std()
    
    # Calculate Wins and Losses
    wins = len(nba[(nba['HomeTeam'] == team) & (nba['HomePTS'] > nba['AwayPTS'])]) + len(nba[(nba['AwayTeam'] == team) & (nba['AwayPTS'] > nba['HomePTS'])])
    losses = len(nba[(nba['HomeTeam'] == team) & (nba['HomePTS'] < nba['AwayPTS'])]) + len(nba[(nba['AwayTeam'] == team) & (nba['AwayPTS'] < nba['HomePTS'])])

    # Append calculated values to the dictionary
    teams_dict['Team'].append(team)
    teams_dict['Wins'].append(wins)
    teams_dict['Losses'].append(losses)
    teams_dict['OFFH'].append(home_offense_mean)
    teams_dict['DEFH'].append(home_defense_mean)
    teams_dict['OFFA'].append(away_offense_mean)
    teams_dict['DEFA'].append(away_defense_mean)
    teams_dict['OFFHsd'].append(home_offense_std)
    teams_dict['DEFHsd'].append(home_defense_std)
    teams_dict['OFFAsd'].append(away_offense_std)
    teams_dict['DEFAsd'].append(away_defense_std)

# Create a dataframe from the dictionary
teams = pd.DataFrame(teams_dict).sort_values(by='Wins', ascending=False)

merged_df = pd.merge(teams, teams_abb, left_on='Team', right_on='abbreviation', how='left')
merged_df['Team'] = merged_df['team']

merged_df.drop('team', axis=1, inplace=True)
merged_df.drop('abbreviation', axis=1, inplace=True)

teams = merged_df

## Game simulation function

In [113]:
def simulate_game(home, away, teams):
    max_overtimes = 7
    overtime_multiplier = 5/48

    home_stats = teams[teams['Team'] == home].iloc[0]
    away_stats = teams[teams['Team'] == away].iloc[0]

    homeOFFH = home_stats['OFFH']
    homeOFFHsd = home_stats['OFFHsd']
    homeDEFH = home_stats['DEFH']
    homeDEFHsd = home_stats['DEFHsd']

    awayOFFA = away_stats['OFFA']
    awayOFFAsd = away_stats['OFFAsd']
    awayDEFA = away_stats['DEFA']
    awayDEFAsd = away_stats['DEFAsd']

    # Simulating Quarters
    quarters = {'H_Q1': [], 'A_Q1': [], 'H_Q2': [], 'A_Q2': [], 'H_Q3': [], 'A_Q3': [], 'H_Q4': [], 'A_Q4': []}
    for i in range(1, 5):
        home_quarter = int(round(((random.gauss(homeOFFH, homeOFFHsd) + random.gauss(awayDEFA, awayDEFAsd)) / 2) / 4, 0))
        away_quarter = int(round(((random.gauss(awayOFFA, awayOFFAsd) + random.gauss(homeDEFH, homeDEFHsd)) / 2) / 4, 0))
        quarters[f'H_Q{i}'].append(home_quarter)
        quarters[f'A_Q{i}'].append(away_quarter)

    # Calculating scores
    home_scores = sum(quarters['H_Q1']) + sum(quarters['H_Q2']) + sum(quarters['H_Q3']) + sum(quarters['H_Q4'])
    away_scores = sum(quarters['A_Q1']) + sum(quarters['A_Q2']) + sum(quarters['A_Q3']) + sum(quarters['A_Q4'])

    # Handling overtime if necessary
    home_OT = 0
    away_OT = 0
    num_overtimes = 0
    if home_scores == away_scores:
        num_overtimes = 0
        while home_scores == away_scores and num_overtimes < max_overtimes:
            num_overtimes += 1
            home_OT += int(round(((random.gauss(homeOFFH, homeOFFHsd) + random.gauss(awayDEFA, awayDEFAsd)) / 2) * overtime_multiplier, 0))
            away_OT += int(round(((random.gauss(awayOFFA, awayOFFAsd) + random.gauss(homeDEFH, homeDEFHsd)) / 2) * overtime_multiplier, 0))
            home_scores += home_OT
            away_scores += away_OT

    # Creating the DataFrame
    data = {
        'HomeTeam': [home],
        'AwayTeam': [away],
        'HomePTS': [home_scores],
        'AwayPTS': [away_scores],
        'H_Q1': [sum(quarters['H_Q1'])],
        'A_Q1': [sum(quarters['A_Q1'])],
        'H_Q2': [sum(quarters['H_Q2'])],
        'A_Q2': [sum(quarters['A_Q2'])],
        'H_Q3': [sum(quarters['H_Q3'])],
        'A_Q3': [sum(quarters['A_Q3'])],
        'H_Q4': [sum(quarters['H_Q4'])],
        'A_Q4': [sum(quarters['A_Q4'])],
        'H_OT': [home_OT],
        'A_OT': [away_OT],
        '#_OT': [num_overtimes]
    }
    return pd.DataFrame(data)

# Simulating a match

In [146]:
game = simulate_game('Houston Rockets', 'Golden State Warriors', teams)
game

,HomeTeam,AwayTeam,HomePTS,AwayPTS,H_Q1,A_Q1,H_Q2,A_Q2,H_Q3,A_Q3,H_Q4,A_Q4,H_OT,A_OT,#_OT
0,Houston Rockets,Golden State Warriors,108,113,28,27,28,28,26,28,26,30,0,0,0


# Simulating a match 1000 times

In [151]:
games = []
iterations = 10000

for _ in range(iterations):
    game = simulate_game('Houston Rockets', 'Golden State Warriors', teams)
    games.append(game)

montecarlo = pd.concat(games, ignore_index=True)
montecarlo

,HomeTeam,AwayTeam,HomePTS,AwayPTS,H_Q1,A_Q1,H_Q2,A_Q2,H_Q3,A_Q3,H_Q4,A_Q4,H_OT,A_OT,#_OT
0,Houston Rockets,Golden State Warriors,109,107,27,30,27,25,27,27,28,25,0,0,0
1,Houston Rockets,Golden State Warriors,113,107,27,32,26,25,30,23,30,27,0,0,0
2,Houston Rockets,Golden State Warriors,105,112,26,23,27,30,25,31,27,28,0,0,0
3,Houston Rockets,Golden State Warriors,103,107,27,26,24,26,29,29,23,26,0,0,0
4,Houston Rockets,Golden State Warriors,114,104,31,25,27,27,28,27,28,25,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Houston Rockets,Golden State Warriors,112,106,31,26,26,30,27,27,28,23,0,0,0
9996,Houston Rockets,Golden State Warriors,111,110,28,26,27,25,28,32,28,27,0,0,0
9997,Houston Rockets,Golden State Warriors,117,118,27,29,30,24,26,24,24,30,10,11,1
9998,Houston Rockets,Golden State Warriors,117,110,30,27,32,23,30,31,25,29,0,0,0
